# Домашнє завдання: Прогнозування орендної плати за житло

## Мета завдання
Застосувати знання з лекції для побудови моделі лінійної регресії, що прогнозує орендну плату за житло в Індії. Ви пройдете весь цикл вирішення задачі машинного навчання: від дослідницького аналізу до оцінки якості моделі.

## Опис датасету
**House Rent Prediction Dataset** містить інформацію про 4700+ оголошень про оренду житла в Індії з такими параметрами:
- **BHK**: Кількість спалень, залів, кухонь
- **Rent**: Орендна плата (цільова змінна)
- **Size**: Площа в квадратних футах
- **Floor**: Поверх та загальна кількість поверхів
- **Area Type**: Тип розрахунку площі
- **Area Locality**: Район
- **City**: Місто
- **Furnishing Status**: Стан меблювання
- **Tenant Preferred**: Тип орендаря
- **Bathroom**: Кількість ванних кімнат
- **Point of Contact**: Контактна особа

---

## Завдання 1: Завантаження та перший огляд даних (1 бал)

**Що потрібно зробити:**
1. Завантажте дані з файлу `House_Rent_Dataset.csv`
2. Виведіть розмір датасету
3. Покажіть перші 5 рядків
4. Виведіть загальну інформацію про дані (включно з типами даних та кількістю значень)


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
data_path = "drive/MyDrive/Homeworks/House_Rent_Dataset.csv"
df = pd.read_csv(data_path)

In [4]:
df.shape

(4746, 12)

In [5]:
df.iloc[:5]

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


## Завдання 2: Дослідницький аналіз даних (EDA) (5 балів)

**Що потрібно зробити:**
1. **Аналіз пропущених значень.** Перевірте наявність і відсоток пропущених значень у кожній колонці
2. **Базова статистика.** Обчисліть базову статистику (середнє, квартилі, стандартне відхилення) для числових змінних.
3. **Аналіз цільової змінної.** Побудуйте гістограму розподілу цільової змінної (Rent)
4. **Робота з викидами.** Знайдіть та видаліть викиди в цільовій змінній (якщо є). Визначити викиди можна будь-яким зрозумілим для вас способом, як варіант - таким, що використовується в побудові box-plot (https://en.wikipedia.org/wiki/Box_plot#Example_with_outliers).
5. **Аналіз категоріальних змінних.** Виведіть кількість унікальних значень для кожної з категоріальних колонок.


In [7]:
df.columns = df.columns.str.strip()

In [8]:
missing_count = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
print(missing_count)
print(missing_percent)

Posted On            0
BHK                  0
Rent                 0
Size                 0
Floor                0
Area Type            0
Area Locality        0
City                 0
Furnishing Status    0
Tenant Preferred     0
Bathroom             0
Point of Contact     0
dtype: int64
Posted On            0.0
BHK                  0.0
Rent                 0.0
Size                 0.0
Floor                0.0
Area Type            0.0
Area Locality        0.0
City                 0.0
Furnishing Status    0.0
Tenant Preferred     0.0
Bathroom             0.0
Point of Contact     0.0
dtype: float64


No missing values in the dataset.

In [9]:
stats = df[['BHK', 'Rent', 'Size', 'Bathroom']].describe()
stats.round(2)

,BHK,Rent,Size,Bathroom
count,4746.00,4746.00,4746.00,4746.00
mean,2.08,34993.45,967.49,1.97
std,0.83,78106.41,634.20,0.88
min,1.00,1200.00,10.00,1.00
25%,2.00,10000.00,550.00,1.00
50%,2.00,16000.00,850.00,2.00
75%,3.00,33000.00,1200.00,2.00
max,6.00,3500000.00,8000.00,10.00


In [10]:
fig = px.histogram(
    df,
    x='Rent',
    nbins=100,
    title='Rent Distribution',
    labels={'Rent': 'Rent','Count': 'Count'})

fig.update_layout(showlegend=False,height=400, title_x=0.5)

fig.show();

In [11]:
df['log_rent'] = np.log1p(df['Rent'])

fig = px.histogram(
    df,
    x='log_rent',
    nbins=100,
    title='Rent Distribution (Log Rent - target variable)',
    labels={'log_rent': 'Log Rent','count': 'count'})

fig.update_layout(showlegend=False,height=400,title_x=0.5)

fig.show();

Data appears to be more of a normal distribution. There are several distinct peaks (between 9 and 10) and some outliers (near 7 and 15), most likely related to cheap and luxury flat listings respectively.

In [12]:
# Prior to removing outliers
fig = px.box(
    df,
    y="log_rent",
    title="Box-plot prior to removing outliers",
    labels={'log_rent': 'Log Rent'})

fig.update_layout(height=500, width=500,title_x=0.5)

fig.show();

In [13]:
Q1 = df['log_rent'].quantile(0.25)
Q3 = df['log_rent'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_final = df[(df['log_rent'] >= lower_bound) & (df['log_rent'] <= upper_bound)]
print(f"Rows removed: {len(df) - len(df_final)}")

Rows removed: 116


In [14]:
fig = px.box(df_final, y="log_rent", title="Box-plot after removing outliers")
fig.update_layout(height=500, width=500,title_x=0.5)

fig.show();

In [15]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
unique_counts = df[categorical_cols].nunique().sort_values(ascending=False)
print(unique_counts)

Area Locality        2235
Floor                 480
Posted On              81
City                    6
Area Type               3
Furnishing Status       3
Tenant Preferred        3
Point of Contact        3
dtype: int64



## Завдання 3: Аналіз кореляцій та взаємозв'язків (3 бали)

**Що потрібно зробити:**
1. Обчисліть матрицю кореляцій для числових змінних
2. Візуалізуйте кореляційну матрицю за допомогою heatmap
3. Побудуйте scatter plot між Size та Rent
4. Проаналізуйте взаємозв'язок між BHK та Rent за допомогою boxplot (який розподіл плати для різних значень BHK)


In [16]:
numeric_df = df_final.select_dtypes(include=['int64', 'float64'])
print(numeric_df)

      BHK   Rent  Size  Bathroom   log_rent
0       2  10000  1100         2   9.210440
1       2  20000   800         1   9.903538
2       2  17000  1000         1   9.741027
3       2  10000   800         1   9.210440
4       2   7500   850         1   8.922792
...   ...    ...   ...       ...        ...
4741    2  15000  1000         2   9.615872
4742    3  29000  2000         3  10.275086
4743    3  35000  1750         3  10.463132
4744    3  45000  1500         2  10.714440
4745    2  15000  1000         2   9.615872

[4630 rows x 5 columns]


In [17]:
corr_matrix = numeric_df.corr()
print(corr_matrix)

               BHK      Rent      Size  Bathroom  log_rent
BHK       1.000000  0.465588  0.709501  0.783854  0.544741
Rent      0.465588  1.000000  0.445408  0.582143  0.882235
Size      0.709501  0.445408  1.000000  0.708872  0.502644
Bathroom  0.783854  0.582143  0.708872  1.000000  0.646430
log_rent  0.544741  0.882235  0.502644  0.646430  1.000000


In [18]:
fig = px.imshow(
    corr_matrix,
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale='RdBu_r',
    title="Correlation matrix",
    labels=dict(color="Correlation"))

fig.update_layout(width=500, height=500, title_x=0.5, margin=dict(l=50, r=50, t=100, b=50))

fig.show();

In [19]:
fig = px.scatter(
    df_final,
    x="Size",
    y="Rent",
    title="Relationship between Property Size (sq ft) and Rent",
    labels={"Size": "Size (sq ft.)", "Rent": "Rent Amount"},
    trendline="ols",
    opacity=0.6)

fig.update_layout(width=1000,height=700,title_x=0.5)

fig.show();

Positive correlation is observed as the trendline goes up.

In [20]:
fig = px.scatter(
    df_final,
    x="Size",
    y="log_rent",
    title="Relationship between Property Size (sq. ft) and Rent",
    labels={"Size": "Size (sq. ft)", "Rent": "log_rent"},
    trendline="ols",
    opacity=0.6)

fig.update_layout(width=1000,height=700,title_x=0.5)

fig.show();

In [21]:
fig = px.scatter(
    df_final,
    x="Size",
    y="log_rent",
    color="City",
    title="Relationship between Property Size (sq. ft) and Rent",
    labels={"Size": "Size (sq.ft)", "Rent": "log_rent"},
    trendline="ols",
    opacity=0.6)

fig.update_layout(width=1000,height=700,title_x=0.5)

fig.show();

In [22]:
fig = px.box(
    df_final,
    x="BHK",
    y="Rent",
    title=" Distribution of rent amount depending on the number of rooms (Bedroom Hall Kitchen(BHK))",
    labels={"BHK": "Number of rooms (BHK)", "Rent": "Rent Amount"},
    color="BHK")

fig.update_layout(width=900,height=600, showlegend=False, title_x=0.5)

fig.show();

In [23]:
fig = px.box(
    df_final,
    x="BHK",
    y="log_rent",
    title="Distribution of rent amount depending on the number of rooms (Bedroom Hall Kitchen (BHK))",
    labels={"BHK": "Number of rooms (BHK)", "Rent": "log_rent"},
    color="BHK")

fig.update_layout(width=900,height=600, showlegend=False, title_x=0.5)

fig.show();

The median price is growing steadily from 1 to 4BHK meaning that the number of rooms is a stable factor in pricing. Anomalies are observed for 5 and 6BHK. The largest number of outliers is observed in 2BHK meaning that these flats are popular and vary significantly in pricing range (from cheap to expensive ones).

## Завдання 4: Feature Engineering та підготовка даних (4 бали)

**Що потрібно зробити:**
1. Закодуйте категоріальні змінні за допомогою One-Hot Encoding. Пригадайте, що в лекції ми говорили щодо кодування кат. змінних з великої кількістю різних значень і як працювати з такими випадками. Ви можете закодувати не всі кат. змінні, а лише ті, що вважаєте за потрібні (скажімо ті, що мають відносно небагато різних значень).
2. **Опціонально (по 0.5 бала за кожну доцільну ознаку):** Додайте нові ознаки, обчислені на основі наявних даних, які б на ваш погляд були корисними для моделі
3. Виберіть ознаки для побудови моделі (виключіть непотрібні колонки). Виключити можна, наприклад, ті колонки, які мають категоріальний тип і забагато (більше 20) різних значень. Треба виключити хоча б 1 колонку.
4. Розділіть дані на ознаки (X) та цільову змінну (y)
5. Застосуйте стандартизацію до числових ознак


In [24]:
cat_columns = df.select_dtypes(include='object').columns
for col in cat_columns:
    print(col, ":", df[col].nunique(), "unique numbers")

Posted On : 81 unique numbers
Floor : 480 unique numbers
Area Type : 3 unique numbers
Area Locality : 2235 unique numbers
City : 6 unique numbers
Furnishing Status : 3 unique numbers
Tenant Preferred : 3 unique numbers
Point of Contact : 3 unique numbers


In [25]:
cols_encode = ['Area Type', 'Furnishing Status', 'Tenant Preferred', 'City', 'Point of Contact']
df_encoded = pd.get_dummies(df, columns=cols_encode, drop_first=True)

In [26]:
df_encoded.head()

,Posted On,BHK,Rent,Size,Floor,Area Locality,Bathroom,log_rent,Area Type_Carpet Area,Area Type_Super Area,...,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2022-05-18,2,10000,1100,Ground out of 2,Bandel,2,9.210440,False,True,...,True,True,False,False,False,False,True,False,False,True
1,2022-05-13,2,20000,800,1 out of 3,"Phool Bagan, Kankurgachi",1,9.903538,False,True,...,False,True,False,False,False,False,True,False,False,True
2,2022-05-16,2,17000,1000,1 out of 3,Salt Lake City Sector 2,1,9.741027,False,True,...,False,True,False,False,False,False,True,False,False,True
3,2022-07-04,2,10000,800,1 out of 2,Dumdum Park,1,9.210440,False,True,...,True,True,False,False,False,False,True,False,False,True
4,2022-05-09,2,7500,850,1 out of 2,South Dum Dum,1,8.922792,True,False,...,True,False,False,False,False,False,True,False,False,True


In [27]:
df_encoded['Room_Size_Ratio'] = df['Size'] / df['BHK']
df_encoded['Bath_Per_BHK'] = df['Bathroom'] / df['BHK']

In [28]:
df_encoded.head(3)

,Posted On,BHK,Rent,Size,Floor,Area Locality,Bathroom,log_rent,Area Type_Carpet Area,Area Type_Super Area,...,Tenant Preferred_Family,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Point of Contact_Contact Builder,Point of Contact_Contact Owner,Room_Size_Ratio,Bath_Per_BHK
0,2022-05-18,2,10000,1100,Ground out of 2,Bandel,2,9.210440,False,True,...,False,False,False,False,True,False,False,True,550.0,1.0
1,2022-05-13,2,20000,800,1 out of 3,"Phool Bagan, Kankurgachi",1,9.903538,False,True,...,False,False,False,False,True,False,False,True,400.0,0.5
2,2022-05-16,2,17000,1000,1 out of 3,Salt Lake City Sector 2,1,9.741027,False,True,...,False,False,False,False,True,False,False,True,500.0,0.5


In [29]:
cols_to_drop = ['Posted On', 'Area Locality', 'Rent', 'Floor']
X = df_encoded.drop(columns=cols_to_drop + ['log_rent'])
y = df_encoded['log_rent']
X.columns.tolist()

['BHK',
 'Size',
 'Bathroom',
 'Area Type_Carpet Area',
 'Area Type_Super Area',
 'Furnishing Status_Semi-Furnished',
 'Furnishing Status_Unfurnished',
 'Tenant Preferred_Bachelors/Family',
 'Tenant Preferred_Family',
 'City_Chennai',
 'City_Delhi',
 'City_Hyderabad',
 'City_Kolkata',
 'City_Mumbai',
 'Point of Contact_Contact Builder',
 'Point of Contact_Contact Owner',
 'Room_Size_Ratio',
 'Bath_Per_BHK']

In [30]:
print(f"X: {X.shape}")
print(f"y: {y.shape}")
print("\nFeatures to be included in model:")
print(X.columns.tolist())

X: (4746, 18)
y: (4746,)

Features to be included in model:
['BHK', 'Size', 'Bathroom', 'Area Type_Carpet Area', 'Area Type_Super Area', 'Furnishing Status_Semi-Furnished', 'Furnishing Status_Unfurnished', 'Tenant Preferred_Bachelors/Family', 'Tenant Preferred_Family', 'City_Chennai', 'City_Delhi', 'City_Hyderabad', 'City_Kolkata', 'City_Mumbai', 'Point of Contact_Contact Builder', 'Point of Contact_Contact Owner', 'Room_Size_Ratio', 'Bath_Per_BHK']


In [31]:
from sklearn.preprocessing import StandardScaler
num_features = ['BHK', 'Size', 'Bathroom', 'Room_Size_Ratio','Bath_Per_BHK']
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[num_features] = scaler.fit_transform(X[num_features])

print(X_scaled[num_features].describe().round(2))

           BHK     Size  Bathroom  Room_Size_Ratio  Bath_Per_BHK
count  4746.00  4746.00   4746.00          4746.00       4746.00
mean      0.00     0.00     -0.00            -0.00         -0.00
std       1.00     1.00      1.00             1.00          1.00
min      -1.30    -1.51     -1.09            -2.15         -2.37
25%      -0.10    -0.66     -1.09            -0.51          0.09
50%      -0.10    -0.19      0.04            -0.04          0.09
75%       1.10     0.37      0.04             0.43          0.09
max       4.71    11.09      9.08            35.64         29.67


## Завдання 5: Розділення даних та навчання моделі (3 бали)

**Що потрібно зробити:**
1. Розділіть дані на навчальну (80%) та тестову (20%) вибірки.
2. Створіть модель лінійної регресії.
3. Навчіть модель на навчальних даних.
4. Виведіть усі коефіцієнти моделі (ваги) та напишіть, які 2 ознаки найбільше впливають на прогноз.
5. Зробіть прогнози на тренувальній та тестовій вибірках.

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2,random_state=42)

print(f"X_train: {X_train.shape[0]} records")
print(f"X_test: {X_test.shape[0]} records")

X_train: 3796 records
X_test: 950 records


In [33]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

print("Model successfylly trained!")

Model successfylly trained!


In [34]:
print("Model Coefficients:")
for feature, weight in zip(model.feature_names_in_, model.coef_):
    print(f"{feature}: {weight:.4f}")

print(f"\nIntercept: {model.intercept_:.4f}")

Model Coefficients:
BHK: 0.1227
Size: 0.1699
Bathroom: 0.2762
Area Type_Carpet Area: 0.1260
Area Type_Super Area: 0.0870
Furnishing Status_Semi-Furnished: -0.1699
Furnishing Status_Unfurnished: -0.2837
Tenant Preferred_Bachelors/Family: -0.0549
Tenant Preferred_Family: -0.1368
City_Chennai: -0.0369
City_Delhi: 0.1875
City_Hyderabad: -0.1555
City_Kolkata: -0.2892
City_Mumbai: 1.0074
Point of Contact_Contact Builder: -0.4322
Point of Contact_Contact Owner: -0.3645
Room_Size_Ratio: 0.0838
Bath_Per_BHK: -0.1047

Intercept: 10.0984


*   City_Mumbai (1.0074): the rent amount for the flat located in Mumbai will be higher than the baseline.
*   Point of Contact_Contact Builder (-0.4322): renting through a Builder is associated with a significantly lower price.




In [35]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("First 5 predictions on the test set (log_rent):")
print(y_test_pred[:5])

First 5 predictions on the test set (log_rent):
[ 9.54251102  9.64959755 10.43159702 10.31595791 10.9444236 ]


In [36]:
y_test_actual_money = np.exp(y_test)
y_test_pred_money = np.exp(y_test_pred)

print(f"Actual price: {y_test_actual_money.values[0]:.0f}")
print(f"Model prediction: {y_test_pred_money[0]:.0f}")

Actual price: 16001
Model prediction: 13940


In [37]:
comparison_df = pd.DataFrame({'Actual Rent': y_test_actual_money.values[:10],'Predicted Rent': y_test_pred_money[:10]})

comparison_df['Difference'] = comparison_df['Actual Rent'] - comparison_df['Predicted Rent']
comparison_df['Error %'] = (comparison_df['Difference'].abs() / comparison_df['Actual Rent']) * 100

print("Comparison for the first 10 properties:")
print(comparison_df.round(0))

Comparison for the first 10 properties:
   Actual Rent  Predicted Rent  Difference  Error %
0      16001.0         13940.0      2061.0     13.0
1      12001.0         15516.0     -3515.0     29.0
2      28001.0         33914.0     -5913.0     21.0
3       8001.0         30211.0    -22210.0    278.0
4      46001.0         56637.0    -10636.0     23.0
5      17001.0         19554.0     -2553.0     15.0
6      57001.0         64260.0     -7259.0     13.0
7       9501.0         12965.0     -3464.0     36.0
8     400001.0        285224.0    114777.0     29.0
9      15001.0         12208.0      2793.0     19.0


## Завдання 6: Оцінка якості моделі (2 бали)

**Що потрібно зробити:**
1. Обчисліть MAE, RMSE та R² для навчальної та тестової вибірок
2. Порівняйте метрики та зробіть висновок про якість моделі
3. Проаналізуйте і дайте висновок, чи є ознаки перенавчання або недонавчання (**Нагадування**: перенавчання - коли модель дуже добре працює на тренувальних даних, але погано на тестових; недонавчання - коли модель погано працює навіть на тренувальних даних)
4. Побудуйте графік розсіювання "реальні vs прогнозовані значення" та зробіть висновок про якість моделі


In [38]:
from sklearn import metrics

y_train_actual = np.exp(y_train)
y_train_pred_money = np.exp(model.predict(X_train))

y_test_actual = np.exp(y_test)
y_test_pred_money = np.exp(model.predict(X_test))

def print_money_metrics(y_true, y_pred, label):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    r2 = metrics.r2_score(y_true, y_pred)

    print(f"--- Metrics {label} ---")
    print(f"R²: {r2:.4f}")
    print(f"MAE: {mae:.0f}")
    print(f"RMSE: {rmse:.0f}")
    print("-" * 40)

print_money_metrics(y_train_actual, y_train_pred_money, "TRAIN")
print_money_metrics(y_test_actual, y_test_pred_money, "TEST")

--- Metrics TRAIN ---
R²: 0.3567
MAE: 12464
RMSE: 65300
----------------------------------------
--- Metrics TEST ---
R²: 0.7381
MAE: 11285
RMSE: 32308
----------------------------------------


*   Train R²: 0.3567. Test R²:  0.7381. R² indicates that the model explains 73% variance in the data.
*   Train MAE is 12464. Test MAE is 11285. The model is stable and makes an error of about 12000.
*   Train RMSE: 65300 is influenced most likely by outliers.
*   A low R² on the training set tells us that the model is underfitting most likely due to extreme values.

In [39]:
train_analysis = pd.DataFrame({'Actual': y_train_actual, 'Pred': y_train_pred_money})
train_analysis['Error'] = (train_analysis['Actual'] - train_analysis['Pred']).abs()

print("The largest errors in the TRAIN set worsening the metrics:")
print(train_analysis.sort_values(by='Error', ascending=False).head(5))

The largest errors in the TRAIN set worsening the metrics:
         Actual           Pred         Error
1837  3500001.0   63672.987961  3.436328e+06
827   1000001.0  272698.460323  7.273025e+05
3656   600001.0   13761.199209  5.862398e+05
4185   200001.0  686472.469628  4.864715e+05
1329   850001.0  368734.159154  4.812668e+05


In [40]:
df.loc[[1837]]

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,log_rent
1837,2022-06-08,3,3500000,2500,4 out of 4,Carpet Area,Marathahalli,Bangalore,Semi-Furnished,Bachelors,3,Contact Agent,15.068274


In [41]:
df.loc[[4185]]

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,log_rent
4185,2022-06-06,1,200000,8000,Ground out of 4,Super Area,"Beeramguda, Ramachandra Puram, NH 9",Hyderabad,Unfurnished,Bachelors/Family,10,Contact Owner,12.206078


In [42]:
plot_df = pd.DataFrame({'Actual Rent': y_test_actual,'Predicted Rent': y_test_pred_money})


fig = px.scatter(
    plot_df,
    x='Actual Rent',
    y='Predicted Rent',
    title='Actual Rent vs Predicted (Test Set)',
    labels={'Actual Rent': 'Actual Rent', 'Predicted Rent': 'Predicted'},
    opacity=0.6,
    trendline="ols",
    trendline_color_override="red")


fig.add_shape(
    type="line", line=dict(dash="dash", color="green", width=2),
    x0=plot_df['Actual Rent'].min(), y0=plot_df['Actual Rent'].min(),
    x1=plot_df['Actual Rent'].max(), y1=plot_df['Actual Rent'].max())

fig.show();

The red regression line is clearly below the ideal diagonal (green dashed line) at higher rent levels. The model consistently underpredicts expensive properties. The model works well for normal flats (less than 150k).


## Завдання 7: Аналіз помилок (4 бали)

**Що потрібно зробити:**
1. Обчисліть помилки (residuals = реальні - прогнозовані значення)
2. Побудуйте гістограму розподілу помилок
3. Створіть scatter plot помилок відносно величини прогнозованих значень. Чи росте помилка з ростом прогнозованого значення?
4. Знайдіть 5 прогнозів з найбільшими помилками
5. Проаналізуйте, на яких типах житла модель помиляється найбільше. Типи можна розрізняти за кількістю кімнат чи містом, наприклад.
6. Подумайте і напишіть, які наступні кроки ви б зробили, аби поліпшити якість моделі. Опціонально можна їх зробити і ми перевіримо :)

In [43]:
residuals = y_test_actual - y_test_pred_money
res_analysis = pd.DataFrame({'Actual': y_test_actual,'Predicted': y_test_pred_money,'Residual': residuals})

print("Residuals Statistics:")
print(res_analysis['Residual'].describe())

Residuals Statistics:
count       950.000000
mean       3074.853914
std       32178.423426
min     -232737.781181
25%       -3573.435213
50%        -296.927997
75%        4030.838097
max      597836.359616
Name: Residual, dtype: float64


In [44]:
res_analysis

,Actual,Predicted,Residual
1566,16001.0,13939.907143,2061.092857
3159,12001.0,15515.542650,-3514.542650
538,28001.0,33914.469100,-5913.469100
2630,8001.0,30210.895352,-22209.895352
4418,46001.0,56637.330951,-10636.330951
...,...,...,...
4124,18001.0,14894.171105,3106.828895
3400,25001.0,16237.870100,8763.129900
1941,14001.0,12999.067244,1001.932756
3679,26001.0,18378.702324,7622.297676


In [45]:
fig = px.histogram(
    x=residuals,
    nbins=50,
    title='Residuals Distribution',
    labels={'x': 'Error (Actual - Predicted)', 'Count': 'Count'},
    color_discrete_sequence=['blue'])

fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="Ideal prediction")
fig.update_layout(height=400)
fig.show();

In [46]:
fig = px.scatter(
    x=y_test_pred_money,
    y=residuals,
    title='Residuals: Error vs Prediction',
    labels={'x': 'Predicted', 'y': 'Actual - Predicted'},
    opacity=0.6,
    trendline="ols",
    trendline_color_override="red")

fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.show();

The model performs relatively well for medium-priced apartments.
It becomes unstable and much less accurate for expensive ones.
Extreme values are strongly influencing the model and worsening metrics.
The error grows with higher predicted values.

In [47]:
analysis_df = pd.DataFrame({'Actual': y_test_actual,'Predicted': y_test_pred_money}, index=y_test.index)
analysis_df['Error'] = (analysis_df['Actual'] - analysis_df['Predicted']).abs()

top_5_errors = analysis_df.sort_values(by='Error', ascending=False).head(5)

top_5_full = top_5_errors.join(df[['BHK', 'Size', 'City', 'Furnishing Status']])

print("5 predictions with largest errors:")
print(top_5_full)

5 predictions with largest errors:
         Actual      Predicted          Error  BHK  Size       City  \
1001  1200001.0  602164.640384  597836.359616    4  5000     Mumbai   
3148   330001.0   90179.398510  239821.601490    3  3600    Chennai   
2846   150001.0  382738.781181  232737.781181    4  4000      Delhi   
4457   400001.0  616305.061971  216304.061971    4  7000  Hyderabad   
1718   380001.0  168001.671786  211999.328214    4  3500  Bangalore   

     Furnishing Status  
1001    Semi-Furnished  
3148    Semi-Furnished  
2846    Semi-Furnished  
4457    Semi-Furnished  
1718    Semi-Furnished  


*   The size - all top errors involve properties ranging from 3,500 to 7,000 sqft.
*  Geographic Anomalies - in Mumbai the model failed to predict the price of 1.2 million. This city has the a super-luxury segment (beyond the reach of simple linear model). In Delhi the prediction overestimated the rent price.
*   All of the largest errors (except one) involve 4-bedroom properties.





1. Handling extreme outliers.
2. Create a seperate model for a luxury segment (segmenting market).
3. Adding more interection terms like City x Size or City x BHK (as flat size behaves differently across cities as well as the rent for 4bedroom in Mumbai does not equal to 4bedroom in Delhi).
